# **Création de la Database Argent**

## **Imports et Dataframes**

### **Imports et création de la database**

In [3]:
import pandas as pd
import os

from sqlalchemy import create_engine 

In [4]:
engine = create_engine("sqlite:///db_argent.sqlite", echo=True)